In [1]:
import getpass
import os

os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_API_KEY'] = getpass.getpass()
os.environ['OPENAI_API_KEY'] = getpass.getpass()

from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model='deepseek-chat', 
    openai_api_key=os.environ['OPENAI_API_KEY'], 
    openai_api_base='https://api.deepseek.com'
)

In [7]:
from typing import Optional

from langchain_core.pydantic_v1 import BaseModel, Field


class Joke(BaseModel):
    """Joke to tell user."""

    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline to the joke")
    rating: Optional[int] = Field(description="How funny the joke is, from 5 to 10")


structured_llm = model.with_structured_output(Joke)

structured_llm.invoke("Tell me a joke about cats")

# ! output nothing
# with the help of LangSmith, we find that the model replies a joke but `PydanticToolsParser` returns `null`
# this means `with_structured_output` is only responsible for parsing output and do nothing in input phase

In [8]:
# so we should define format in prompt
structured_llm = model.with_structured_output(Joke, method="json_mode")
structured_llm.invoke(
    "Tell me a joke about cats, respond in JSON with `setup`, `punchline` and `rating` keys"
)
# this time, a json reply can be parsed to `Joke` correctly
# be attention, we said `rating` should be 5-10 in `Joke` but model doesn't know this

Joke(setup='Why did the cat sit on the computer?', punchline='To keep an eye on the mouse!', rating=4)

In [11]:
# so we should provide more information in prompt
# few-shot prompting: giving several examples to define format and requirements
from langchain_core.prompts import ChatPromptTemplate

system = """You are a game reviewer. Your specialty is reviewing video games. \
Return a review for game which has the description (no more than 30 words) and the rating (from 5 to 10).

Here are some examples:

example_user: Zelda: Breath of the Wild
example_assistant: {{"description": "An open-world masterpiece that combines stunning visuals, intricate puzzles, and dynamic combat, offering unparalleled freedom and exploration.", "rating": 10}}

example_user: DotA2
example_assistant: {{"description": "A highly strategic and competitive MOBA with deep mechanics, diverse heroes, and a thriving esports scene, offering endless replayability.", "rating": 9.5}}

example_user: JX3
example_assistant: {{"description": "A visually stunning MMORPG set in ancient China, featuring rich storytelling, intricate martial arts combat, and a vibrant, immersive world.", ""rating": 9}}"""

prompt = ChatPromptTemplate.from_messages([("system", system), ("human", "{input}")])

class Game(BaseModel):
    """Review of the video game"""

    description: str = Field(description="The description of the game")
    rating: int = Field(description="How good the game is, from 5 to 10")

structured_llm = model.with_structured_output(Game, method="json_mode")

few_shot_structured_llm = prompt | structured_llm
few_shot_structured_llm.invoke("Genshin Impact")

Game(description='A vibrant open-world RPG with engaging elemental combat, diverse characters, and a captivating story, offering both solo and cooperative play.', rating=9)

In [12]:
# as a common thought, we want `PydanticToolsParser` to do prompt for us
# yes, it can, but the auto-generated prompt is not so good as few-shot prompting
from langchain_core.output_parsers import PydanticOutputParser

parser = PydanticOutputParser(pydantic_object=Game)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a game reviewer. Your specialty is reviewing video games. Return a review for game which has the description (no more than 30 words) and the rating (from 5 to 10). Wrap the output in `json` tags\n{format_instructions}",
        ),
        ("human", "{query}"),
    ]
).partial(format_instructions=parser.get_format_instructions())

query = "Genshin Impact"

print(prompt.invoke(query).to_string())

System: You are a game reviewer. Your specialty is reviewing video games. Return a review for game which has the description (no more than 30 words) and the rating (from 5 to 10). Wrap the output in `json` tags
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"description": "Review of the video game", "properties": {"description": {"title": "Description", "description": "The description of the game", "type": "string"}, "rating": {"title": "Rating", "description": "How good the game is, from 5 to 10", "type": "integer"}}, "required": ["description", "rating"]}
```
Human: Genshin Impact


In [13]:
chain = prompt | model | parser
chain.invoke({"query": query})

Game(description='Engaging open-world adventure with stunning visuals and a deep, immersive storyline.', rating=9)